In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# A - Préparation de datasets (train et test)
# A-1- Extraction des datasets
# A-1-1- Extrain de dataset train.7z

!pip install py7zr
!python -m py7zr x ../input/cifar-10/train.7z /kaggle/working/

In [ ]:
# A-1-2- Extrain de dataset test.7z
!python -m py7zr x ../input/cifar-10/test.7z /kaggle/working/

In [ ]:
# A-2- Création de dataset
import glob
import os
import cv2
import numpy as np 

   #nombre d'images de train data
train_images_path = "/kaggle/working/train"
num_of_train_images= len(glob.glob(train_images_path+"/*"))
print("Le nombre d'images de train data est",num_of_train_images)

   #nombre d'images de test data
test_images_path = "/kaggle/working/test"    
num_of_test_images= len(glob.glob(test_images_path+"/*"))
print("Le nombre d'images de test data est", num_of_test_images)

    # Ajouter les images de train  au dataset créée
train_images = [[]]*num_of_train_images
for dir_name, _, filenames in os.walk('/kaggle/working/train'):
    for filename in filenames:
        image_index = int(filename.split(".")[0])-1
        img = cv2.imread(os.path.join(dir_name,filename))
        train_images[image_index] = img
        
     # Ajouter les images de test  au dataset créée
test_images = [[]]*num_of_test_images
for dir_name, _, filenames in os.walk('/kaggle/working/test'):
    for filename in filenames:
        image_index = int(filename.split(".")[0])-1
        img = cv2.imread(os.path.join(dir_name,filename))
        test_images[image_index] = img

In [ ]:
    # normalisation des datasets 
train_images = np.asarray(train_images, dtype=float)/255
test_images = np.asarray(test_images, dtype=float)/255
train_images[0]
   #Enregistrement des datasets sous forme de matrices
np.save("../working/train_images.npy", train_images)
np.save("../working/test_images.npy", test_images)

In [ ]:
     #Vérifier le nombre d'image dans chaque data
print("train data:",train_images.shape)
print("test data:",test_images.shape)
        

In [ ]:
      # Affichage des dix premières images
import matplotlib.pyplot as plt
import pandas as pd
train_labels=pd.read_csv("../input/cifar-10/trainLabels.csv")
fig=plt.figure(figsize=(8, 8))
columns = 5
rows = 2
for i in range(1, columns*rows +1):
    img = train_images[i]
    fig.add_subplot(rows, columns, i, title=[train_labels.label[i]])
    plt.imshow(img)
plt.show()

In [ ]:
    # L'augmentation de datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
datagen = ImageDataGenerator(
        rotation_range=0.3,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=True)

In [ ]:
# B - Création du modèle
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, datasets,models,layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, DepthwiseConv2D, BatchNormalization
from tensorflow.keras.models import load_model

# B- 1- Création du modèle
model=Sequential()

# B-2- Création des couches 
       # La couche de convolution 1
model.add(Conv2D(32, (3, 3), activation='relu', strides=(1,1), padding='same', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu, depth_multiplier=3))
model.add(Dropout(rate =0.1))
    
    
model.add(Conv2D(64, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu))
model.add(Dropout(rate = 0.1))
    
model.add(Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu))
model.add(Dropout(rate = 0.4))
    
model.add(Conv2D(128, (3, 3), activation='relu', strides=(1, 1), padding='same'))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=(1,1), strides=(1, 1), padding='same', activation=keras.activations.relu))
    
    
model.add(Conv2D(256, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=(3,3), strides=(1, 1), padding='same', activation=keras.activations.relu))
    
    
    
model.add(Conv2D(512, (1, 1), activation='relu', strides=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(DepthwiseConv2D(kernel_size=(1,1), strides=(1, 1), padding='same', activation=keras.activations.relu))
model.add(Dropout(rate = 0.4))
    
model.add(Flatten())
model.add(Dropout(rate = 0.3))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(rate = 0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate = 0.4))
model.add(Dense(10, activation='softmax'))




In [ ]:
 # Afficher un résumé
model.summary()

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
# C- L'entraînement du modèle 
labels_dict =  {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
labels_dict_reversed = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
train_labels['Category'] = train_labels.label.map(labels_dict_reversed)
train_labels_categories = keras.utils.to_categorical(train_labels.Category, num_classes)#.astype('uint8')
x_train, x_val, y_train, y_val = train_test_split(train_images, train_labels_categories, random_state=0, test_size=0.05)    
    
    
    # Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # Conserver le meilleur modèle
mcp_save = ModelCheckpoint('../working/best_model', save_best_only=True, monitor='val_accuracy', mode='max')
    # Entraînement du modèle
    
num_of_epochs = 5
batch_size = 64 
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), validation_data=(x_val, y_val), callbacks=[mcp_save], epochs=num_of_epochs)